view for v1 predictor

In [2]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

/home/srjeon/miniconda3/envs/ag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df = pd.read_parquet("../data/HD_data/train_v1.parquet")
test_df = pd.read_parquet("../data/HD_data/test_v1.parquet")

In [5]:
predictor = TabularPredictor.load("./AutogluonModels/ag-20231018_024855/")

In [6]:
print(predictor.leaderboard(silent=True))

                     model  score_val  pred_time_val     fit_time  \
0      WeightedEnsemble_L3 -43.542377     180.717363  4218.588755   
1    NeuralNetTorch_BAG_L2 -43.551134     178.562527  4138.465751   
2      WeightedEnsemble_L2 -44.868450      41.971762  1518.937651   
3    NeuralNetTorch_BAG_L1 -45.079861       1.863544  1304.953166   
4           XGBoost_BAG_L2 -46.414749     177.808346  3332.064069   
5           XGBoost_BAG_L1 -50.862259       1.874161    71.474507   
6          LightGBM_BAG_L2 -50.884777     176.566256  3271.635758   
7     LightGBMLarge_BAG_L2 -50.918639     176.659594  3280.806809   
8          CatBoost_BAG_L2 -51.081054     175.983907  3275.651306   
9        LightGBMXT_BAG_L2 -51.157292     176.631081  3272.467091   
10  NeuralNetFastAI_BAG_L2 -51.817703     179.765471  4141.101819   
11    ExtraTreesMSE_BAG_L2 -51.993095     216.807722  3484.934431   
12    ExtraTreesMSE_BAG_L1 -52.923289      40.225160   213.489757   
13  RandomForestMSE_BAG_L2 -53.182

In [7]:
predictor.feature_importance(train_df)

These features in provided data are not utilized by the predictor and will be ignored: ['ARI_CO_VE', 'ARI_PO_CXL1', 'ARI_PO_DEJ2', 'ARI_PO_DIN2', 'ARI_PO_EFG4', 'ARI_PO_GIW5', 'ARI_PO_GQJ7', 'ARI_PO_IVU2', 'ARI_PO_JEA6', 'ARI_PO_KIU2', 'ARI_PO_LHD1', 'ARI_PO_ONW1', 'ARI_PO_PDL4', 'ARI_PO_QXE2', 'ARI_PO_RGT8', 'ARI_PO_XZF6', 'ARI_PO_ZME5', 'FLAG_Algeria', 'FLAG_Argentina', 'FLAG_Azerbaijan', 'FLAG_Bahrain', 'FLAG_Bermuda', 'FLAG_Brunei', 'FLAG_Bulgaria', 'FLAG_Cambodia', 'FLAG_Chile', 'FLAG_Comoros False', 'FLAG_Cook Islands', 'FLAG_Curacao', 'FLAG_Djibouti', 'FLAG_Dominica', 'FLAG_Egypt', 'FLAG_Equatorial Guinea', 'FLAG_Equatorial Guinea False', 'FLAG_Estonia', 'FLAG_Ethiopia', 'FLAG_Faeroes (Fas)', 'FLAG_France', 'FLAG_France (Fis)', 'FLAG_Gabon', 'FLAG_Gambia', 'FLAG_Georgia', 'FLAG_Ghana', 'FLAG_Gibraltar', 'FLAG_Guinea-Bissau', 'FLAG_Guyana', 'FLAG_Guyana False', 'FLAG_Honduras', 'FLAG_Iran', 'FLAG_Irish Republic', 'FLAG_Jamaica', 'FLAG_Jordan', 'FLAG_Kiribati', 'FLAG_Kuwait', 'FLA

,importance,stddev,p_value,n,p99_high,p99_low
DIST,35.012865,0.934475,6.083153e-08,5,36.936961,33.088768
SHIP_TYPE_CATEGORY_Container,7.367271,0.538590,3.403300e-06,5,8.476235,6.258306
weekday,6.817949,0.303983,4.729499e-07,5,7.443855,6.192043
ARI_CO_CN,5.436279,0.188310,1.724954e-07,5,5.824013,5.048545
DEADWEIGHT,4.544371,0.356514,4.508545e-06,5,5.278438,3.810304
...,...,...,...,...,...,...
FLAG_Finland,-0.000584,0.004050,6.184318e-01,5,0.007754,-0.008923
ARI_PO_FFM2,-0.001036,0.032709,5.265390e-01,5,0.066313,-0.068385
FLAG_United Arab Emirates,-0.001153,0.002669,8.056762e-01,5,0.004342,-0.006649
ARI_CO_PE,-0.005920,0.012410,8.268993e-01,5,0.019631,-0.031471


In [8]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_df, model=model_to_use)

/home/srjeon/miniconda3/envs/ag/lib/python3.10/site-packages/catboost/core.py:1222: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, pairs, weight,


In [9]:
model_pred

0          3.438760
1         17.373907
2         24.205833
3         63.411274
4          0.007266
            ...    
220486    72.538940
220487    67.750481
220488    28.058092
220489     0.007266
220490    48.084354
Name: CI_HOUR, Length: 220491, dtype: float32

In [10]:
submit = pd.read_csv("../data/HD_data/sample_submission.csv")
submit["CI_HOUR"] = [i if i > 0 else 0.0 for i in model_pred]
submit.to_csv("csv/autogluon_v1.csv", index=False)